In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.8 MB/s eta 0:00:00


In [3]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="praw")

import logging
# Suppress PRAW warnings
logging.getLogger("praw").setLevel(logging.CRITICAL)

import praw
import pandas as pd
import time
from datetime import datetime

# Reddit API credentials
reddit = praw.Reddit(
    client_id='E2jh-fCEPm1SQC01GQcVOQ',
    client_secret='65oz9bU3v2ns6V_7j-fJ7gS8dA077g',
    user_agent='InterstellarScraper/1.0'
)

# 1) Find all subreddits with "interstellar" in their name
movie_title = "Interstellar"
found_subs = []
for sub in reddit.subreddits.search(movie_title, limit=50):
    if movie_title.lower() in sub.display_name.lower():
        found_subs.append(sub.display_name)

print(f"Found {len(found_subs)} subreddits for '{movie_title}': {found_subs}")

# 2) Scrape settings
posts_per_type = 20000    # limit per category ('hot','new','top')
post_types = ['hot', 'new', 'top']

all_rows = []
sno = 1

# 3) Iterate, scrape, and continue on error
for subreddit_name in found_subs:
    count = 0
    print(f"\n--- Scraping r/{subreddit_name} ---")
    try:
        subreddit = reddit.subreddit(subreddit_name)
        for pt in post_types:
            if pt == 'hot':
                posts = subreddit.hot(limit=posts_per_type)
            elif pt == 'new':
                posts = subreddit.new(limit=posts_per_type)
            else:
                posts = subreddit.top(limit=posts_per_type)

            for post in posts:
                text = f"{post.title}\n\n{post.selftext}"
                date_str = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                all_rows.append({
                    'SNO': sno,
                    'SUBREDDIT_NAME': subreddit_name,
                    'TEXT': text,
                    'DATE': date_str
                })
                sno += 1
                count += 1

            # be kind to Reddit’s API
            time.sleep(2)

        print(f"Completed r/{subreddit_name} — obtained {count} posts.")

    except Exception as e:
        print(f"Error scraping r/{subreddit_name}: {e}")
        # move on to next subreddit
        continue

# 4) Save to CSV
df = pd.DataFrame(all_rows, columns=['SNO', 'SUBREDDIT_NAME', 'TEXT', 'DATE'])
output_file = 'interstellar_posts.csv'
df.to_csv(output_file, index=False)

print(f"\nDone! Total posts collected: {len(df)}. Saved to '{output_file}'.")
print(df.head(10))


Found 12 subreddits for 'Interstellar': ['interstellar', 'InterstellarArt', 'InterstellarMemes', 'InterstellarPilots', 'InterstellarMarines', 'InterstellarRift', 'InterstellarGarage', 'Interstellar_Blends', 'InterstellarTransport', 'Project_Interstellar', 'Interstellar_NYE', 'InterstellarWar']

--- Scraping r/interstellar ---
Completed r/interstellar — obtained 2890 posts.

--- Scraping r/InterstellarArt ---
Completed r/InterstellarArt — obtained 2838 posts.

--- Scraping r/InterstellarMemes ---
Completed r/InterstellarMemes — obtained 357 posts.

--- Scraping r/InterstellarPilots ---
Completed r/InterstellarPilots — obtained 2734 posts.

--- Scraping r/InterstellarMarines ---
Completed r/InterstellarMarines — obtained 963 posts.

--- Scraping r/InterstellarRift ---
Completed r/InterstellarRift — obtained 540 posts.

--- Scraping r/InterstellarGarage ---
Completed r/InterstellarGarage — obtained 1577 posts.

--- Scraping r/Interstellar_Blends ---
Completed r/Interstellar_Blends — obtai

In [4]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="praw")

import logging
# Suppress PRAW warnings
logging.getLogger("praw").setLevel(logging.CRITICAL)

import praw
import pandas as pd
import time
from datetime import datetime

# Reddit API credentials
reddit = praw.Reddit(
    client_id='E2jh-fCEPm1SQC01GQcVOQ',
    client_secret='65oz9bU3v2ns6V_7j-fJ7gS8dA077g',
    user_agent='InterstellarScraper/1.0'
)

# 1) Find all subreddits with "interstellar" in their name
movie_title = "Interstellar"
found_subs = []
for sub in reddit.subreddits.search(movie_title, limit=50):
    if movie_title.lower() in sub.display_name.lower():
        found_subs.append(sub.display_name)

print(f"Found {len(found_subs)} subreddits for '{movie_title}': {found_subs}")

# 2) Scrape settings
posts_per_type = 20000    # limit per category ('hot','new','top')
post_types = ['hot', 'new', 'top']

all_rows = []
sno = 1

# 3) Iterate, scrape, continue on error
for subreddit_name in found_subs:
    count = 0
    print(f"\n--- Scraping r/{subreddit_name} ---")
    try:
        subreddit = reddit.subreddit(subreddit_name)
        for pt in post_types:
            if pt == 'hot':
                posts = subreddit.hot(limit=posts_per_type)
            elif pt == 'new':
                posts = subreddit.new(limit=posts_per_type)
            else:
                posts = subreddit.top(limit=posts_per_type)

            for post in posts:
                text = f"{post.title}\n\n{post.selftext}"
                date_str = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                upvotes = getattr(post, 'ups', None)
                downvotes = getattr(post, 'downs', None)
                all_rows.append({
                    'SNO': sno,
                    'SUBREDDIT_NAME': subreddit_name,
                    'TEXT': text,
                    'DATE': date_str,
                    'UPVOTES': upvotes,
                    'DOWNVOTES': downvotes
                })
                sno += 1
                count += 1

            # Respect rate limits
            time.sleep(2)

        print(f"Completed r/{subreddit_name} — obtained {count} posts.")

    except Exception as e:
        print(f"Error scraping r/{subreddit_name}: {e}")
        continue

# 4) Save to CSV
df = pd.DataFrame(all_rows, columns=[
    'SNO', 'SUBREDDIT_NAME', 'TEXT', 'DATE', 'UPVOTES', 'DOWNVOTES'
])
output_file = 'interstellar_posts_with_votes.csv'
df.to_csv(output_file, index=False)

print(f"\nDone! Total posts collected: {len(df)}. Saved to '{output_file}'.")
print(df.head(10))


Found 12 subreddits for 'Interstellar': ['interstellar', 'InterstellarArt', 'InterstellarMemes', 'InterstellarPilots', 'InterstellarMarines', 'InterstellarRift', 'InterstellarGarage', 'Interstellar_Blends', 'InterstellarTransport', 'Project_Interstellar', 'Interstellar_NYE', 'InterstellarWar']

--- Scraping r/interstellar ---
Completed r/interstellar — obtained 2890 posts.

--- Scraping r/InterstellarArt ---
Completed r/InterstellarArt — obtained 2838 posts.

--- Scraping r/InterstellarMemes ---
Completed r/InterstellarMemes — obtained 357 posts.

--- Scraping r/InterstellarPilots ---
Completed r/InterstellarPilots — obtained 2734 posts.

--- Scraping r/InterstellarMarines ---
Completed r/InterstellarMarines — obtained 963 posts.

--- Scraping r/InterstellarRift ---
Completed r/InterstellarRift — obtained 540 posts.

--- Scraping r/InterstellarGarage ---
Completed r/InterstellarGarage — obtained 1577 posts.

--- Scraping r/Interstellar_Blends ---
Completed r/Interstellar_Blends — obtai